In [2]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from bayes_opt import BayesianOptimization

In [7]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','effort']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)
dataset = cocomo

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Prepare the data.
X = dataset.drop(columns=['effort'])  # Features
y = dataset['effort']  # Target (effort)

# Initialize K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# Initialize lists to store RMSE and MMRE for each fold
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Standardize the features to have zero mean and unit variance.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train_scaled, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test_scaled)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate the Root Mean Squared Error (RMSE) for this fold.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)

     # Convert y_pred and y_test to NumPy arrays for MMRE calculation
    y_pred = np.array(y_pred).flatten()
    y_test = np.array(y_test)
    
    # Calculate the Mean Magnitude of Relative Error (MMRE) for this fold.
    mmre = mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

Epoch 1/10
7/7 [==============================] - 0s 2ms/step - loss: 4596197.0000
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 4595847.5000
Epoch 3/10
7/7 [==============================] - 0s 999us/step - loss: 4595529.0000
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 4595239.0000
Epoch 5/10
7/7 [==============================] - 0s 833us/step - loss: 4594951.5000
Epoch 6/10
7/7 [==============================] - 0s 1ms/step - loss: 4594759.5000
Epoch 7/10
7/7 [==============================] - 0s 1ms/step - loss: 4594512.0000
Epoch 8/10
7/7 [==============================] - 0s 999us/step - loss: 4594274.0000
Epoch 9/10
7/7 [==============================] - 0s 1ms/step - loss: 4594110.5000
Epoch 10/10
1/1 [==============================] - 0s 35ms/step
Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 1276428.1250
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 1276225.8750
Epoch 3/10
7/7 [=